In [1]:
#import packages
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

In [2]:
#import data
df_crime = pd.read_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/MergedData_2017-2021.csv')
df_covid_ward = pd.read_excel(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/DC-COVID-19-Data-for-11-5-2021-2.xlsx'
                              , sheet_name = 'Total Cases by Ward', engine='openpyxl')
df_covid_death = pd.read_excel(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/DC-COVID-19-Data-for-11-5-2021-2.xlsx'
                               , sheet_name = 'Lives Lost by Ward', engine='openpyxl')
df_sick_police = pd.read_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/daily_count_police_out_due_to_covid.csv')


In [3]:
#delete columns not needed from crime data
del df_crime['REPORT_DAT']
del df_crime['BLOCK']
del df_crime['YBLOCK']
del df_crime['XBLOCK']
del df_crime['LATITUDE']
del df_crime['LONGITUDE']
del df_crime['END_DATE']

df_crime = df_crime.rename(columns = {'START_DATE':'date'})

df_crime['date'] = pd.to_datetime(df_crime['date'], utc=True)
df_crime.dtypes

SHIFT                    object
METHOD                   object
OFFENSE                  object
WARD                    float64
DISTRICT                float64
date        datetime64[ns, UTC]
dtype: object

In [4]:
df_crime.dtypes

SHIFT                    object
METHOD                   object
OFFENSE                  object
WARD                    float64
DISTRICT                float64
date        datetime64[ns, UTC]
dtype: object

In [5]:
#covid case data
df_covid_ward = df_covid_ward.transpose()
df_covid_ward.reset_index(drop = True, inplace= True)
df_covid_ward.columns=('date', 'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5', 'ward_6'
                      , 'ward_7', 'ward_8', 'unknown')
df_covid_ward = df_covid_ward.iloc[1: , :]
df_covid_ward['date'] = pd.to_datetime(df_covid_ward['date'], utc=True)
df_covid_ward['total_cases'] = (df_covid_ward['ward_1'] + df_covid_ward['ward_2'] + df_covid_ward['ward_3'] + df_covid_ward['ward_4'] + df_covid_ward['ward_5'] + df_covid_ward['ward_6'] +df_covid_ward['ward_7'] + df_covid_ward['ward_8'] +df_covid_ward['unknown'])
df_covid_ward['daily_cases'] = df_covid_ward['total_cases'].diff(1)

del df_covid_ward['ward_1']
del df_covid_ward['ward_2']
del df_covid_ward['ward_3']
del df_covid_ward['ward_4']
del df_covid_ward['ward_5']
del df_covid_ward['ward_6']
del df_covid_ward['ward_7']
del df_covid_ward['ward_8']
del df_covid_ward['unknown']


df_covid_ward.dtypes

date           datetime64[ns, UTC]
total_cases                 object
daily_cases                 object
dtype: object

In [6]:
#death data
df_covid_death = df_covid_death.transpose()
df_covid_death.reset_index(drop = True, inplace= True)
df_covid_death.columns=('date', 'all', 'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5', 'ward_6'
                      , 'ward_7', 'ward_8', 'Experienced Homelessness', 'unknown', 'na')
df_covid_death = df_covid_death.iloc[1: , :]
df_covid_death['date'] = pd.to_datetime(df_covid_death['date'], utc=True)
df_covid_death['daily_death'] = df_covid_death['all'].diff(1)

del df_covid_death['ward_1']
del df_covid_death['ward_2']
del df_covid_death['ward_3']
del df_covid_death['ward_4']
del df_covid_death['ward_5']
del df_covid_death['ward_6']
del df_covid_death['ward_7']
del df_covid_death['ward_8']
del df_covid_death['unknown']
del df_covid_death['na']
del df_covid_death['Experienced Homelessness']
del df_covid_death['all']

df_covid_death.dtypes

date           datetime64[ns, UTC]
daily_death                 object
dtype: object

In [7]:
#sick police
df_sick_police = df_sick_police.rename(columns = {'DATE_REPORTED':'date'})
df_sick_police['date'] = pd.to_datetime(df_sick_police['date'], utc=True)

df_sick_police.dtypes

date                             datetime64[ns, UTC]
OUT_POSITIVE_QUARANTINE_PSMPD                float64
dtype: object

In [10]:
merged = df_crime.merge(df_covid_ward, how='left', on = 'date')
merged2 = merged.merge(df_covid_death, how='left', on = 'date')
merged3 = merged2.merge(df_sick_police, how='left', on = 'date')
merged3.loc[merged3['total_cases'] >1]

,SHIFT,METHOD,OFFENSE,WARD,DISTRICT,date,total_cases,daily_cases,daily_death,OUT_POSITIVE_QUARANTINE_PSMPD
101927,EVENING,OTHERS,THEFT F/AUTO,2.0,2.0,2020-09-17 00:00:00+00:00,14852,62,0,45.0
104289,EVENING,OTHERS,THEFT F/AUTO,6.0,1.0,2020-11-06 00:00:00+00:00,17891,99,2,87.0
105309,DAY,OTHERS,BURGLARY,1.0,3.0,2020-11-12 00:00:00+00:00,18666,159,0,103.0
105631,DAY,OTHERS,THEFT/OTHER,1.0,3.0,2020-08-30 00:00:00+00:00,13992,33,1,71.0
111896,EVENING,OTHERS,THEFT/OTHER,1.0,3.0,2020-04-07 00:00:00+00:00,1440,229,NaN,315.0
...,...,...,...,...,...,...,...,...,...,...
149518,DAY,OTHERS,THEFT F/AUTO,2.0,3.0,2021-04-15 00:00:00+00:00,46449,134,3,18.0
149950,MIDNIGHT,OTHERS,THEFT F/AUTO,1.0,3.0,2021-04-07 00:00:00+00:00,45498,170,1,25.0
150170,MIDNIGHT,OTHERS,THEFT F/AUTO,6.0,1.0,2021-01-18 00:00:00+00:00,34259,226,4,190.0
150544,EVENING,GUN,ASSAULT W/DANGEROUS WEAPON,6.0,1.0,2021-01-07 00:00:00+00:00,31107,357,1,182.0


In [9]:
merged3.to_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/merged.csv')